In [ ]:
import requests
import pymysql
import time
import json

# Configuración API
URL_areas = 'https://cramer.buk.cl/api/v1/chile/organization/areas/?status=both'
TOKEN = "Xegy8dVsa1H8SFfojJcwYtDL"

# Configuración BD
DB_HOST = "localhost"
DB_USER = "root"
DB_PASSWORD = "cancionanimal"
DB_NAME = "prueba_buk"

print("🔧 Configuración cargada correctamente")

In [ ]:
def obtener_todas_las_areas():
    """
    Obtiene todas las áreas desde la API con paginación y las devuelve filtradas
    con todos los campos necesarios.
    """
    headers = {"auth_token": TOKEN}
    areas_filtradas = []
    url_actual = URL_areas
    pagina_actual = 1
    
    print("🚀 Comenzando la obtención de todas las áreas con paginación...")
    
    while url_actual:
        print(f"📄 Obteniendo página {pagina_actual}...")
        
        try:
            respuesta = requests.get(url_actual, headers=headers)
            respuesta.raise_for_status()
            
            respuesta_api = respuesta.json()
            areas_pagina = respuesta_api['data']
            pagination_info = respuesta_api['pagination']
            
            # Filtrar cada área de esta página
            for area_completa in areas_pagina:
                area_filtrada = {
                    "id": area_completa.get("id"),
                    "name": area_completa.get("name"),
                    "address": area_completa.get("address"),
                    "first_level_id": area_completa.get("first_level_id"),
                    "first_level_name": area_completa.get("first_level_name"),
                    "second_level_id": area_completa.get("second_level_id"),
                    "second_level_name": area_completa.get("second_level_name"),
                    "cost_center": area_completa.get("cost_center"),
                    "status": area_completa.get("status"),
                    "city": area_completa.get("city"),
                }
                areas_filtradas.append(area_filtrada)
            
            print(f"✅ Página {pagina_actual}: {len(areas_pagina)} áreas procesadas")
            print(f"📊 Total acumulado: {len(areas_filtradas)} áreas filtradas")
            
            # Obtener la URL de la siguiente página
            url_actual = pagination_info.get('next')
            pagina_actual += 1
            
            # Pequeña pausa para no sobrecargar la API
            time.sleep(0.5)
            
        except requests.exceptions.RequestException as e:
            print(f"❌ Error en la petición: {e}")
            break
            
    print(f"🎉 ¡Paginación completada! Total de áreas filtradas: {len(areas_filtradas)}")
    return areas_filtradas

# Obtener todas las áreas
areas_filtradas = obtener_todas_las_areas()

In [ ]:
print (json.dumps(areas_filtradas, indent=4, ensure_ascii=False))

In [ ]:
print("🚀 Conectando a MySQL...")
conexion = pymysql.connect(
    host=DB_HOST,
    user=DB_USER,
    password=DB_PASSWORD,
    charset='utf8mb4'
)
cursor = conexion.cursor()

# Usar la base de datos existente
cursor.execute(f"USE {DB_NAME}")

print(f"✅ Conectado a MySQL y usando la base: {DB_NAME}")

print("🚀 Creando tabla areas si no existe...")
sql_create_table = """
CREATE TABLE IF NOT EXISTS areas (
    id INT PRIMARY KEY,
    name VARCHAR(255),
    address TEXT,
    first_level_id INT,
    first_level_name VARCHAR(255),
    second_level_id INT,
    second_level_name VARCHAR(255),
    cost_center VARCHAR(100),
    status VARCHAR(50),
    city VARCHAR(100)
);
"""
cursor.execute(sql_create_table)
print("✅ Tabla 'areas' creada exitosamente")

In [ ]:
print("🚀 Insertando TODAS las áreas en la tabla SQL...")
contador = 0
errores = 0

for area in areas_filtradas:
    try:
        sql = """
        INSERT INTO areas (
            id, name, address, first_level_id, first_level_name,
            second_level_id, second_level_name, cost_center, status, city
        )
        VALUES (
            %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
        )
        ON DUPLICATE KEY UPDATE
            name=VALUES(name),
            address=VALUES(address),
            first_level_id=VALUES(first_level_id),
            first_level_name=VALUES(first_level_name),
            second_level_id=VALUES(second_level_id),
            second_level_name=VALUES(second_level_name),
            cost_center=VALUES(cost_center),
            status=VALUES(status),
            city=VALUES(city)
        """
        
        values = (
            area.get("id"),
            area.get("name"),
            area.get("address"),
            area.get("first_level_id"),
            area.get("first_level_name"),
            area.get("second_level_id"),
            area.get("second_level_name"),
            area.get("cost_center"),
            area.get("status"),
            area.get("city")
        )
        
        cursor.execute(sql, values)
        contador += 1
        
        # Mostrar progreso cada 50 registros
        if contador % 50 == 0:
            print(f"📝 Procesadas {contador} áreas...")
            
    except Exception as error:
        print(f"⚠️ Error insertando área {area.get('id', 'N/A')}: {error}")
        errores += 1

conexion.commit()
print(f"✅ {contador} registros insertados/actualizados en MySQL.")
if errores > 0:
    print(f"⚠️ Se produjeron {errores} errores durante la inserción.")

In [ ]:
cursor.close()
conexion.close()
print("✅ Conexión cerrada correctamente.")